# ibm_sql_query_cpd

Run arbitrary SQL based data transformation jobs on CSV, PARQUET, JSON, AVRO and ORC data stored on Cloud Object Storage using IBM SQL Query. Transformation results are written stored back to Cloud Object Storage

In [ ]:
import os

#os.environ['create_image']='True'
os.environ['repository']='us.icr.io/ai-lifecycle-staging'
os.environ['version']='0.64'
#
#os.environ['install_requirements']='True'

In [2]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-39
    RUN pip install ipython nbformat numpy ibm-cos-sdk-core ibm-cos-sdk ibm-watson-machine-learning ibm-watson-studio-pipelines ibmcloudsql pyyaml
    ADD ibm-sql-query-cpd.py .
    ADD start.sh .
    RUN echo {} > ./.version
    """.format(os.environ['version'])
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    start_file="""
    #!/bin/bash
    echo "Parameter 1: $1"
    echo "Parameter 2: $2"
    echo "Parameter 3: $3"
    echo "Parameter 4: $4"
    echo "Parameter 5: $5"
    echo "Parameter 6: $6"
    echo "Parameter 7: $7"
    echo "Parameter 8: $8"
    echo "Parameter 9: $9"
    echo "Parameter 10: ${10}"
    echo "Parameter 11: ${11}"
    echo "Parameter 12: ${12}"
    echo "Parameter 13: ${13}"
    echo "Parameter 14: ${14}"
    echo "Parameter 15: ${15}"
    echo "Parameter 16: ${16}"
    echo "Parameter 17: ${17}"
    echo "Parameter 18: ${18}"
    echo "Parameter 19: ${19}"
    echo "Parameter 20: ${20}"
    echo "Version":
    cat ./.version
    cat /opt/user-token/token
    python /opt/app-root/src/ibm-sql-query-cpd.py "$1$2" "$3$4" "$5$6" "$7$8" "$9${10}" "${11}${12}" "${13}${14}" "${15}${16}" "${17}${18}" "${19}${20}"
    """
    with open("start.sh", "w") as text_file:
        text_file.write(start_file)

    !chmod 755 start.sh
    !jupyter nbconvert --to script ibm-sql-query-cpd.ipynb    
    !docker build -t ibm_sql_query_cpd:`echo $version` .
    !docker tag ibm_sql_query_cpd:`echo $version` `echo $repository`/ibm_sql_query_cpd:`echo $version`
    !docker push `echo $repository`/ibm_sql_query_cpd:`echo $version`
    !rm Dockerfile
    !rm ibm-sql-query-cpd.py
    !rm start.sh
elif bool(os.environ.get('install_requirements',False)):
    !pip install numpy ibm-cos-sdk-core ibm-cos-sdk ibm-watson-machine-learning ibm-watson-studio-pipelines ibmcloudsql pyyaml

[NbConvertApp] Converting notebook ibm-sql-query-cpd.ipynb to script
[NbConvertApp] Writing 13698 bytes to ibm-sql-query-cpd.py
Sending build context to Docker daemon  304.6kB
Step 1/5 : FROM registry.access.redhat.com/ubi8/python-39
 ---> 54ab5aa3f6b8
Step 2/5 : RUN pip install ipython nbformat numpy ibm-cos-sdk-core ibm-cos-sdk ibm-watson-machine-learning ibm-watson-studio-pipelines ibmcloudsql pyyaml
 ---> Using cache
 ---> b486348bf686
Step 3/5 : ADD ibm-sql-query-cpd.py .
 ---> 50bbf0c1c143
Step 4/5 : ADD start.sh .
 ---> 2a9b823c7f1d
Step 5/5 : RUN echo 0.64 > ./.version
 ---> Running in 3ad11d3f1544
Removing intermediate container 3ad11d3f1544
 ---> a39bf4fdadb9
Successfully built a39bf4fdadb9
Successfully tagged ibm_sql_query_cpd:0.64
The push refers to repository [us.icr.io/ai-lifecycle-staging/ibm_sql_query_cpd]

a847a38b: Preparing 
22e8833a: Preparing 
2032024f: Preparing 
9271e776: Preparing 
a0c9fb84: Preparing 
12ade9b7: Preparing 
45f59bfc: Preparing 
c6aaa811: Preparin

In [ ]:
import glob
import logging
import ibmcloudsql
from ibmcloudsql import SQLQuery
import json
import os
import shutil
import sys
import re
from ibm_watson_machine_learning import APIClient
from ibm_watson_studio_pipelines import WSPipelines
from ibm_watson_studio_pipelines.cpd_paths import CpdScope, CpdPath
import yaml
import ast

In [ ]:
# COS URL where the results of the SQL job are to be stored
target_dir_path = os.environ.get('target_dir_path')

# Asset name to register for the results written by the SQL job
target_asset_name = os.environ.get('target_asset_name')

# sql statement to execute
sql = os.environ.get('sql')

# (unique) Custom Resource Name (CRN) of IBM SQL Query Service
data_engine_crn = os.environ.get('data_engine_crn')

# default: CSV - (will be generated into according STORED AS … clause in the INTO clause)
format = os.environ.get('format' , 'CSV')

# optional, list of columns to use for partitioning the results of the SQL job, will be generated into according PARTITIONED BY (<columns>) clause in the INTO clause)
partition_columns = os.environ.get('partition_columns','')

# optional, number of objects to store the results of the SQL job in, will be generated into according PARTITIONED INTO <num> OBJECTS clause in INTO clause
number_of_objects = int(os.environ.get('number_of_objects', 0))

# optional, number of rows to be stored in each result object of the SQL job, will be generated into according PARTITIONED EVERY <num> ROWS clause in INTO clause
rows_per_object = int(os.environ.get('rows_per_object', 0))

# default: False, only valid when none of the above partitioning option is specified, produces exactly one object with name specified in target_dir_path, twill be generated into sqlClient.rename_exact_result(jobid) after SQL has run.
exact_name = os.environ.get('exact_name', 'False')

# default: False - will be generated into JOBPREFIX NONE in the INTO clause. Will cause results of previous runs with same output_uri to be overwritten, because no unique sub folder will be created for the result)
no_jobid_folder = os.environ.get('no_jobid_folder', 'False')

# default: output.txt - output file name containing the CPD path of the resulting asset
data_asset = os.environ.get('data_asset','output.txt')


In [ ]:
"""
token = 'eyJraWQiOiIyMDIyMDYxNTA4MjYiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJJQk1pZC0yNzAwMDI1NzNZIiwiaWQiOiJJQk1pZC0yNzAwMDI1NzNZIiwicmVhbG1pZCI6IklCTWlkIiwic2Vzc2lvbl9pZCI6IkMtMjc5YjNlYzctOGMwYy00YzQzLWJiMWMtMGNjMWY5MWI0OGM5Iiwic2Vzc2lvbl9leHBfbWF4IjoxNjU3MTg2MDQ2LCJzZXNzaW9uX2V4cF9uZXh0IjoxNjU3MTA3Njc3LCJqdGkiOiI3ODQxYzkyZS0yNzY3LTRiYjMtYmU5OC01ZjM0MjQ0NzU5NmEiLCJpZGVudGlmaWVyIjoiMjcwMDAyNTczWSIsImdpdmVuX25hbWUiOiJSb21lbyIsImZhbWlseV9uYW1lIjoiS2llbnpsZXIiLCJuYW1lIjoiUm9tZW8gS2llbnpsZXIiLCJlbWFpbCI6InJvbWVvLmtpZW56bGVyQGNoLmlibS5jb20iLCJzdWIiOiJyb21lby5raWVuemxlckBjaC5pYm0uY29tIiwiYXV0aG4iOnsic3ViIjoicm9tZW8ua2llbnpsZXJAY2guaWJtLmNvbSIsImlhbV9pZCI6IklCTWlkLTI3MDAwMjU3M1kiLCJuYW1lIjoiUm9tZW8gS2llbnpsZXIiLCJnaXZlbl9uYW1lIjoiUm9tZW8iLCJmYW1pbHlfbmFtZSI6IktpZW56bGVyIiwiZW1haWwiOiJyb21lby5raWVuemxlckBjaC5pYm0uY29tIn0sImFjY291bnQiOnsiYm91bmRhcnkiOiJnbG9iYWwiLCJ2YWxpZCI6dHJ1ZSwiYnNzIjoiOWIxM2I4NTdhMzIzNDFiNzE2NzI1NWRlNzE3MTcyZjUiLCJpbXNfdXNlcl9pZCI6IjgwODAyMjIiLCJpbXMiOiIyMDMyNDc0In0sImlhdCI6MTY1NzEwMDQ3NywiZXhwIjoxNjU3MTAxNjc3LCJpc3MiOiJodHRwczovL2lhbS5jbG91ZC5pYm0uY29tL2lkZW50aXR5IiwiZ3JhbnRfdHlwZSI6InVybjppYm06cGFyYW1zOm9hdXRoOmdyYW50LXR5cGU6cGFzc2NvZGUiLCJzY29wZSI6ImlibSBvcGVuaWQiLCJjbGllbnRfaWQiOiJieCIsImFjciI6MSwiYW1yIjpbInB3ZCJdfQ.VicviebFD7b-evDMQVee6MTGIEMW27f3V2VovTvXRbmJ0pwUeSwyFp5fXULi8kahkBJQReTjsk3SVtWEB1e_HAiAqLtdeu_OzejBXX7o2eugEBxsNLrFUSGUr_bgMrb9IhDk9NUTBnpppIkItwrMBMZSGLT-9tLqa1cd9SdAjwaKR7O3A9MXxN7toP8Pyn8KgMxAeFls33uRKStOcE9zHHnhmup6JWVpMvkoY-cuR-_ZMv1Bb85rjU9u79wofnpzEoJ7_VGu-DtDWjchD028fpi_FAXwmYbU-7JHSgnfXUCuU-EBTPv2bv5tGxnrxROvc5yFPIKz4nMOy0sE5YPW0w'
target_dir_path='cpd:///projects/e0bce158-a9e4-4be6-a1da-20a04a7770f5/connections/de544f56-05c0-4dda-9e00-06458052c467/files/cos-rkie-sql-engine-test/sql_results'
sql='SELECT * FROM cos://eu-de/claimed-test/data.parquet stored as parquet'
sql_query_crn='crn:v1:bluemix:public:sql-query:us-south:a/9b13b857a32341b7167255de717172f5:f9dd6c9e-b24b-4506-819e-e038c92339e4::'
target_location='cos://eu-de/claimed-test-results/data.parquet'
"""

In [ ]:
for element in sys.argv:
    logging.warning('argv raw ' +  element)

parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))


for parameter in parameters:
    exec(parameter)
    logging.warning('Parameter: ' + parameter)

exact_name = ast.literal_eval(exact_name.capitalize())
no_jobid_folder = ast.literal_eval(no_jobid_folder.capitalize())
number_of_objects = number_of_objects if type(number_of_objects)==int else int(number_of_objects) if len(number_of_objects)>0 else 0
rows_per_object = rows_per_object if type(rows_per_object)==int else int(rows_per_object) if len(rows_per_object)>0 else 0


for parameter in parameters:
    exec("logging.warning('final parameter: ' + str({}))".format(parameter.split('=')[0]))
    exec("logging.warning('final parameter type: ' + str(type({})))".format(parameter.split('=')[0]))

In [ ]:
with open('/opt/user-token/token', 'r') as file:
    token = file.read().rstrip()

    logging.warning('Token: ' + token)
    logging.info('TokenI: ' + token)
    
logging.warning('Token: ' + token)
logging.info('TokenIO: ' + token)

In [ ]:
with open(os.environ.get('CPDCONFIG')) as file:
    cpd_config = yaml.load(file, Loader=yaml.FullLoader)
    for service in cpd_config['services']:
        if service['type'] == 'watson-studio':
            cpdaas_url = service['service']['url']

In [ ]:
target_dir_path_object = CpdPath.from_string(target_dir_path)
logging.warning('target_dir_path_object: {}'.format(target_dir_path_object))

project_id = target_dir_path_object.scope_id()
logging.warning('project_id: {}'.format(project_id))

connection_id = target_dir_path_object.resource_id()
logging.warning('connection_id: {}'.format(connection_id))

#bucket_name = target_dir_path_object.bucket_name()
#logging.warning('bucket_name: {}'.format(bucket_name))

dir_path = target_dir_path_object.file_path()
logging.warning('dir_path: {}'.format(dir_path))

In [ ]:
pipelines_client = WSPipelines.from_token(token, url=cpdaas_url)
wml_credentials = pipelines_client.get_wml_credentials(target_dir_path_object.scope())

In [ ]:
wml_credentials['token'] = token
wml_client = APIClient(wml_credentials)
wml_client.set.default_project(project_id)

In [ ]:
wml_client.connections.get_details(connection_id)

In [ ]:
cos_props = wml_client.connections.get_details(connection_id)

logging.warning('cos_props')
logging.warning(cos_props)

logging.warning(type(cos_props))
bucket_name = cos_props['entity']['properties']['bucket']

logging.warning('bucket_name: {}'.format(bucket_name))

In [ ]:
cos_props = cos_props['entity']['properties']
url = cos_props['url'].split('//')[1]
bucket = cos_props['bucket']
logging.warning(cos_props)
#cos_props = cos_props['credentials']

In [ ]:
"""
import json
url = 'dummy'
bucket = 'dummy'

cos_props = '''
{
  "apikey": "sdsd",
  "cos_hmac_keys": {
    "access_key_id": "sdsdsdsdf3",
    "secret_access_key": "sdsfsdf"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key crn:v1:bluemix:public:cloud-object-storage:global:a/9b13b857a32341b7167255de717172f5:24451914-8910-498e-8279-1851c5dd0328:resource-key:315158d1-a7a4-432f-9e11-c6e245e5011e",
  "iam_apikey_name": "viewer-ed22e594-7a2f-4cda-95af-bc7ca806a7b6",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/9b13b857a32341b7167255de717172f5::serviceid:ServiceId-9d606823-b14e-4633-bc49-8eba77ec3d7f",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/9b13b857a32341b7167255de717172f5:24451914-8910-498e-8279-1851c5dd0328::"
}
'''
"""

In [ ]:
"""
import json
url = 'dummy'
bucket = 'dummy'
cos_props = '''
{
    "name": "torstensbucket",
    "origin_country": "us",
    "owner_id": "IBMid-120000ETQ5",
    "properties": {
      "api_key": "sdfsdf",
      "auth_method": "instanceid_apikey",
      "bucket": "torstensbucket",
      "iam_url": "https://iam.test.cloud.ibm.com/identity/token",
      "resource_instance_id": "crn:v1:staging:public:cloud-object-storage:global:a/bb287b72124612756cf16d897c8c18e5:b273ae5e-c945-4aec-a962-12a5e7d36493::",
      "url": "https://s3.us-west.cloud-object-storage.test.appdomain.cloud"
    }
}
'''
"""

In [ ]:
# cos_props = json.loads(cos_props)
if 'api_key' in  cos_props:
    api_key = cos_props['api_key']
elif 'properties' in  cos_props:
    api_key = cos_props['properties']['api_key']
else:
    assert(False, "Can't handle this")
target_location = "cos://{}/{}/{}".format(url,bucket, target_asset_name)
logging.warning(target_location)
logging.warning(api_key)


In [ ]:
#api_key = 'jtbo_VatEv-Fl5GwPj678DId23iOPx4f6TFdNPxbbMrP'
#sql_query_crn = 'crn:v1:bluemix:public:sql-query:us-south:a/9b13b857a32341b7167255de717172f5:cd3e149d-e183-4162-a392-50bab6c64f47::'
logging.warning('instatiating sql query object with token {} and crn {}'.format(token,data_engine_crn))

sqlClient = SQLQuery(api_key=None, token=token, instance_crn=data_engine_crn)

logging.warning('done')


In [ ]:

sql = sql + ' INTO {}'.format(target_location)



if no_jobid_folder:
    sql = sql + ' JOBPREFIX NONE'

sql = sql + ' STORED AS {}'.format(format)


if len(partition_columns) > 0 or number_of_objects > 0 or rows_per_object > 0:
    sql = sql + ' PARTITIONED '

if len(partition_columns) > 0:
    sql = sql + ' BY ({})'.format(partition_columns)

if number_of_objects > 0:
    sql = sql + ' INTO {} OBJECTS'.format(number_of_objects)

if rows_per_object > 0:
    sql = sql + ' EVERY {} ROWS'.format(rows_per_object)

    
print(sql)

job_id = sqlClient.submit_sql(sql)
sqlClient.wait_for_job(job_id)
job_details = sqlClient.get_job(job_id)
job_status = job_details['status']
print("Job " + job_id + " terminated with status: " + job_status)
if job_status == 'completed':
    if exact_name:
        sqlClient.rename_exact_result(job_id)
        print("Job results written to single result object  " + job_details['resultset_location'])
    else:
        print("Job results written to folder " + job_details['resultset_location'])


In [ ]:
metadata = {
  wml_client.data_assets.ConfigurationMetaNames.NAME: target_asset_name,
  wml_client.data_assets.ConfigurationMetaNames.DESCRIPTION: 'Data asset created for SQL Query result',
  wml_client.data_assets.ConfigurationMetaNames.CONNECTION_ID: connection_id,
  wml_client.data_assets.ConfigurationMetaNames.DATA_CONTENT_NAME: dir_path
}
asset_details = wml_client.data_assets.store(meta_props=metadata)


In [ ]:
new_cpd_path = target_dir_path.split('connections')[0]+'assets/'+asset_details['metadata']['asset_id']
with open(data_asset,'w') as fout:
    fout.write(new_cpd_path)